In [1]:
from pymongo import MongoClient

client = MongoClient("mongodb://localhost:27017")

db = client.sample_mflix
movies = db.movies

print(movies)

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'sample_mflix'), 'movies')


In [4]:
# "star"라는 단어가 포함된 영화제목을 가져오세요
import re # 정규표현식 패턴 만드는 라이브러리

regex=re.compile("star",re.I)  # I -> ignore case : 대문자든 소문자든 무시하고 찾아와라!

for movie in movies.find({"title":regex}):
    print(movie)

{'_id': ObjectId('573a1392f29313caabcdb497'), 'plot': 'A young woman comes to Hollywood with dreams of stardom, but achieves them only with the help of an alcoholic leading man whose best days are behind him.', 'genres': ['Drama'], 'runtime': 111, 'rated': 'NOT RATED', 'cast': ['Janet Gaynor', 'Fredric March', 'Adolphe Menjou', 'May Robson'], 'poster': 'https://m.media-amazon.com/images/M/MV5BMmE5ODI0NzMtYjc5Yy00MzMzLTk5OTQtN2Q3MzgwOTllMTY3XkEyXkFqcGdeQXVyNjc0MzMzNjA@._V1_SY1000_SX677_AL_.jpg', 'title': 'A Star Is Born', 'fullplot': 'Esther Blodgett is just another starry-eyed farm kid trying to break into the movies. Waitressing at a Hollywood party, she catches the eye of alcoholic star Norman Maine, is given a test, and is caught up in the Hollywood glamor machine (ruthlessly satirized). She and her idol Norman marry; but his career abruptly dwindles to nothing', 'languages': ['English'], 'released': datetime.datetime(1937, 4, 27, 0, 0), 'directors': ['William A. Wellman', 'Jack Con

In [13]:
#선생님 해설
star_movies=movies.find({"title":{"$regex":".*star.*"}},{"title":1,"_id":0}) #.*Star.* star 앞뒤에 몇개의 글자가 들어와도 상관없어!
for star_movie in star_movies:
    print("영화제목 :",star_movie["title"])

영화제목 : Marjorie Morningstar
영화제목 : Jesus Christ Superstar
영화제목 : Firestarter
영화제목 : Konec starych casu
영화제목 : Bastard Out of Carolina
영화제목 : Fègi Is a Bastard
영화제목 : Superstar
영화제목 : Firestarter 2: Rekindled
영화제목 : Firestarter 2: Rekindled
영화제목 : Battlestar Galactica
영화제목 : Restart
영화제목 : Bastards of the Party
영화제목 : Bastards
영화제목 : Beauty and the Bastard
영화제목 : Los bastardos
영화제목 : Battlestar Galactica: Razor
영화제목 : Telstar: The Joe Meek Story
영화제목 : Battlestar Galactica: The Plan
영화제목 : Battlestar Galactica: Blood & Chrome
영화제목 : Mad Bastards
영화제목 : Superstar
영화제목 : Radiostars
영화제목 : Bastards


In [16]:
# 각 감독들이 제작한 영화의 수가 25개 이상인 감독들만 찾아서 출력해주세요.
pipeline = [
    {"$group" : {"_id":"$directors","count":{"$sum":1}}},
    {"$match":{"count":{"$gte":25}}}
]
for movie in movies.aggregate(pipeline):
    print(movie)

{'_id': ['Steven Soderbergh'], 'count': 26}
{'_id': ['Werner Herzog'], 'count': 31}
{'_id': ['Alfred Hitchcock'], 'count': 31}
{'_id': ['Ingmar Bergman'], 'count': 25}
{'_id': ['Spike Lee'], 'count': 26}
{'_id': ['Michael Apted'], 'count': 27}
{'_id': ['Steven Spielberg'], 'count': 28}
{'_id': ['John Huston'], 'count': 30}
{'_id': ['Clint Eastwood'], 'count': 27}
{'_id': None, 'count': 265}
{'_id': ['Sidney Lumet'], 'count': 29}
{'_id': ['John Ford'], 'count': 29}
{'_id': ['Martin Scorsese'], 'count': 29}
{'_id': ['Michael Winterbottom'], 'count': 25}
{'_id': ['Robert Altman'], 'count': 27}
{'_id': ['Takashi Miike'], 'count': 33}
{'_id': ['Woody Allen'], 'count': 39}


In [23]:
# 선생님 해설 -> 각각의 감독으로 하려면 배열을 풀어야함 unwind 필요!
director_Pipelines=[
    {"$unwind":"$directors"},
    {"$group":{"_id":"$directors","count":{"$sum":1}}},
    {"$match":{"count":{"$gte":25}}}
]

director_counts = list(movies.aggregate(director_Pipelines))
for director_count in director_counts:
    print(f"감독 : {director_count['_id']}, \n 영화수:{director_count['count']}")


감독 : John Huston, 
 영화수:34
감독 : Jean-Luc Godard, 
 영화수:27
감독 : George Cukor, 
 영화수:29
감독 : Robert Altman, 
 영화수:28
감독 : Werner Herzog, 
 영화수:33
감독 : John Ford, 
 영화수:35
감독 : Takashi Miike, 
 영화수:34
감독 : Johnnie To, 
 영화수:27
감독 : Wim Wenders, 
 영화수:27
감독 : Ingmar Bergman, 
 영화수:25
감독 : Ken Loach, 
 영화수:27
감독 : Steven Spielberg, 
 영화수:29
감독 : Martin Scorsese, 
 영화수:32
감독 : Spike Lee, 
 영화수:28
감독 : Michael Winterbottom, 
 영화수:26
감독 : Ridley Scott, 
 영화수:25
감독 : William Wyler, 
 영화수:26
감독 : Clint Eastwood, 
 영화수:27
감독 : Michael Apted, 
 영화수:29
감독 : Woody Allen, 
 영화수:40
감독 : Alfred Hitchcock, 
 영화수:31
감독 : Mario Monicelli, 
 영화수:29
감독 : Sidney Lumet, 
 영화수:30
감독 : Steven Soderbergh, 
 영화수:28


In [31]:
# 평점을 기준으로 상위 5개의 영화를 찾으세요. 단, 투표를 1000표 이상 받은 영화에 한함.
Pipeline = [
    {"$match":{"imdb.votes":{"$gte":1000}}},
    {"$sort":{"imdb.rating":-1}},
    {"$limit":5},
    {"$project":{"_id":0,"title":1,"imdb.rating":1,"imdb.votes":1}}
]


pipes = movies.aggregate(Pipeline)
for pipe in pipes:
    print(pipe)

{'title': 'Band of Brothers', 'imdb': {'rating': 9.6, 'votes': 183802}}
{'title': 'Planet Earth', 'imdb': {'rating': 9.5, 'votes': 82896}}
{'title': 'The Civil War', 'imdb': {'rating': 9.4, 'votes': 4624}}
{'title': 'The Civil War', 'imdb': {'rating': 9.4, 'votes': 4625}}
{'imdb': {'rating': 9.3, 'votes': 1513145}, 'title': 'The Shawshank Redemption'}


In [38]:
top_rated_movies = list(movies.find({"imdb.votes":{"$gte":1000}}).sort("imdb.rating",-1).limit(5))
for movie in top_rated_movies:
    print(f"영화제목 : {movie['title']},평점 : {movie['imdb']['rating']}점")

영화제목 : Band of Brothers,평점 : 9.6점
영화제목 : Planet Earth,평점 : 9.5점
영화제목 : The Civil War,평점 : 9.4점
영화제목 : The Civil War,평점 : 9.4점
영화제목 : The Shawshank Redemption,평점 : 9.3점
